# Самый базовый алгоритм.


Сначала подключим все нужные библиотеки.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import os
import cv2

from sklearn.linear_model import LogisticRegression
import pandas as pd
import sklearn

# use seaborn plotting defaults
import seaborn as sns; sns.set()

Загрузим датасет и сформируем выборку для решения задачи

In [ ]:
import glob
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


Загрузим тренировочные данные с Google Drive и извлечем номера классов из названий изображений.

In [ ]:
from tqdm import tqdm

dirname = "/gdrive/My Drive/Kaggle/train"

Xtrain, ytrain = [], []

for fname in tqdm(os.listdir(dirname)):
    img = cv2.imread(os.path.join(dirname, fname), cv2.IMREAD_GRAYSCALE)
    ytrain.append(int(fname[-6:-1][0:2]))
        
    h, w = img.shape
    size = min(h, w)
    h0 = int((h - size) / 2)
    w0 = int((w - size) / 2)
        
    img = img[h0: h0 + size, w0: w0 + size]
    img = cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)
        
    Xtrain.append(img)

len(Xtrain), len(ytrain)

100%|██████████| 621/621 [00:02<00:00, 304.36it/s]


(621, 621)

Загрузим тестовые изображения из Google Drive.

Тестовые изображения загружаются в произвольном порядке, поэтому сделаем сортировку по номерам, указанным в названиях изображений.

In [ ]:
def sort_by_index(fname):
  s = fname
  s = s.replace('img', '', 1)
  s = s.replace('.jpg', '', 1)
  str1 = ''
  for i in s:
    if i.isdigit():
      str1 += i
    else:
      break
  return int(str1)


In [ ]:
from tqdm import tqdm

dirname = "/gdrive/My Drive/Kaggle/test"

Xtest = []
Names = []

for fname in tqdm(sorted(os.listdir(dirname), key=sort_by_index)):
    img = cv2.imread(os.path.join(dirname, fname), cv2.IMREAD_GRAYSCALE)
    Names.append(fname)
        
    h, w = img.shape
    size = min(h, w)
    h0 = int((h - size) / 2)
    w0 = int((w - size) / 2)
        
    img = img[h0: h0 + size, w0: w0 + size]
    img = cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)
        
    Xtest.append(img)

len(Xtest)

100%|██████████| 4150/4150 [00:37<00:00, 110.63it/s]


4150

Обучим baseline-модель (логистическую регрессию без дополнительных шагов) и сделаем предсказание.

In [ ]:
from sklearn.linear_model import LogisticRegression

Xtrain = np.asarray([el.ravel() for el in Xtrain])

model_lr = LogisticRegression()
model_lr.fit(Xtrain, ytrain)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [ ]:
Xtest = np.asarray([el.ravel() for el in Xtest])
pred_lr = model_lr.predict(Xtest)

Сохраним предсказание в виде, который требуется в соревновании и сохраним файл с предсказаниями.

In [ ]:
import pandas as pd

pred_df = pd.DataFrame(list(zip(Names, pred_lr)), columns = ['img', 'label'])
pred_df.head()

,img,label
0,img0.jpg,1
1,img1.jpg,17
2,img2.jpg,11
3,img3.jpg,16
4,img4.jpg,16


In [ ]:
pred_df.to_csv("first_submission.csv", index=False)

Результат на Kaggle - 0,39. Хочется получше.

# Улучшим алгоритм с помощью Аугментации и снижения размерности.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import os
import cv2

from sklearn.linear_model import LogisticRegression
import pandas as pd
import sklearn

# use seaborn plotting defaults
import seaborn as sns; sns.set()

In [ ]:
import glob
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
from tqdm import tqdm

dirname = "/gdrive/My Drive/Kaggle/train"

Xtrain, ytrain = [], []

for fname in tqdm(os.listdir(dirname)):
    img = cv2.imread(os.path.join(dirname, fname), cv2.IMREAD_GRAYSCALE)
    ytrain.append(int(fname[-6:-1][0:2]))
        
    h, w = img.shape
    size = min(h, w)
    h0 = int((h - size) / 2)
    w0 = int((w - size) / 2)
        
    img = img[h0: h0 + size, w0: w0 + size]
    img = cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)
        
    Xtrain.append(img)

len(Xtrain), len(ytrain)

100%|██████████| 621/621 [00:01<00:00, 347.79it/s]


(621, 621)

In [ ]:
def sort_by_index(fname):
  s = fname
  s = s.replace('img', '', 1)
  s = s.replace('.jpg', '', 1)
  str1 = ''
  for i in s:
    if i.isdigit():
      str1 += i
    else:
      break
  return int(str1)

In [ ]:
from tqdm import tqdm

dirname = "/gdrive/My Drive/Kaggle/test"

Xtest = []
Names = []

for fname in tqdm(sorted(os.listdir(dirname), key=sort_by_index)):
    img = cv2.imread(os.path.join(dirname, fname), cv2.IMREAD_GRAYSCALE)
    Names.append(fname)
        
    h, w = img.shape
    size = min(h, w)
    h0 = int((h - size) / 2)
    w0 = int((w - size) / 2)
        
    img = img[h0: h0 + size, w0: w0 + size]
    img = cv2.resize(img, (64, 64), interpolation = cv2.INTER_AREA)
        
    Xtest.append(img)

len(Xtest)

100%|██████████| 4150/4150 [00:12<00:00, 332.42it/s]


4150

In [ ]:
import random
import albumentations as A

from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

def get_aug(image):
    angle = np.arange(-10,11,1)
    angle0 = random.choice(angle)

    shift = 0.01*np.arange(-10,11,1)
    shift0 = random.choice(shift)

    transform = A.ShiftScaleRotate(shift_limit=shift0,rotate_limit=angle0,scale_limit=0,p=0.5)
    
    augmented_image = transform(image=image)['image']

    transform = A.HorizontalFlip(p=0.5)

    return transform(image=augmented_image)['image']

In [ ]:
XtrainAug = []
ytrainAug = []

for (a,b) in zip(Xtrain,ytrain):
    for i in range(5):
      XtrainAug.append(get_aug(a))
      ytrainAug.append(b)

In [ ]:
XtrainAug = np.asarray([el.ravel() for el in XtrainAug])
Xtest = np.asarray([el.ravel() for el in Xtest])

In [ ]:
XtrainAug.shape

(6210, 4096)

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA #Principal Components Analysis
from sklearn.pipeline import make_pipeline

pca = KernelPCA(n_components=3105, kernel='poly')
model_lr = LogisticRegression()

model = make_pipeline(pca, model_lr)

model.fit(XtrainAug, ytrainAug)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('kernelpca', KernelPCA(kernel='poly', n_components=3105)),
                ('logisticregression', LogisticRegression())])

In [ ]:
pred_lr = model.predict(Xtest)

In [ ]:
import pandas as pd

pred_df = pd.DataFrame(list(zip(Names, pred_lr)), columns = ['img', 'label'])
pred_df.head()

,img,label
0,img0.jpg,1
1,img1.jpg,17
2,img2.jpg,7
3,img3.jpg,18
4,img4.jpg,16


In [ ]:
pred_df.to_csv("submission3.csv", index=False)

Качество улучшилось и стало составлять 0,72.